In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import emoji
import jaconv


In [2]:
cd ../

C:\Users\井原輝人\Desktop\sorcecode\twitter-compe


In [3]:
#特徴量の抽出
def add_flags_nums(df):
    #urlの有無
    df["is_url"] = df['text'].str.contains(r'(https?://[\w/:%#\$&\?\(\)~\.=\+\-]+)')
    df["is_url"] = (df["is_url"]*1).astype(np.float64)
    
    #locationの有無
    df["is_location"] = df["location"].isnull()
    df["is_url"] = (df["is_url"]*1).astype(np.float64)
    
    #絵文字の個数
    df["num_emoji"] = np.zeros(len(df.index))
    for idx in range(0,len(df.index)):
        for letter in df["text"].loc[idx]:
            if letter in  emoji.EMOJI_UNICODE.values():
                df["num_emoji"].loc[idx] += 1
    df["num_emoji"] = df["num_emoji"].astype(np.float64)    
    
    #リプライの数
    df["num_reply"] = df['text'].str.count('@').astype(np.float64)
    
    #ハッシュタグの数
    df["num_hash"] = df['text'].str.count('#').astype(np.float64)
    
    #テキストの長さ
    df["text_length"] = np.zeros(df.shape[0])
    for idx in range(df.shape[0]):
        df["text_length"].loc[idx] = len(df["text"].loc[idx])
    df["text_length"] = df["text_length"].astype(np.float64)
    
    #生えてる草の個数
    df["num_kusa"] = df["text"].str.count("w")
    df["num_kusa"] = df["num_kusa"].astype(np.float64)
    
    #日付が入っているか
    df["is_date"] = df['text'].str.contains(r'([0-9]日)')
    df["is_date"] = (df["is_date"]*1).astype(np.float64)
    df["is_time"] = df['text'].str.contains(r'([0-9]時)')
    df["is_time"] = (df["is_time"]*1).astype(np.float64)
    
    return df

In [4]:
#テキストのクリーニング
def cleaning(df):
    #urlの削除
    df["text"] = df["text"].replace(r'(https?://[\w/:%#\$&\?\(\)~\.=\+\-]+)',"",regex=True)
    #replyの削除
    df["text"] = df["text"].replace(r'@([A-Za-z0-9_])+',"",regex = True)
    #全角スペースを半角スペースに
    df["text"] = jaconv.z2h(df["text"].str,digit=True, ascii = True) 
    #大文字を小文字に
    df["text"] = df["text"].str.lower()
    #数字を0に置き換える
    df["text"] = df["text"].replace(r'([0-9])',"0",regex = True)    
    #アカウント名と思しきものを消す
    ####
    return df

In [5]:
#分かち書き
from janome.tokenizer import Tokenizer
def wakati(df):
    df["text_wakati"] = str()
    t= Tokenizer()

    for idx in range(0,df.shape[0]):
        df["text_wakati"].loc[idx] = [token.base_form for token in t.tokenize(df["text"].loc[idx])]
    return df

In [6]:
#1文字の単語を消す
def del_one(df):
    delonelist = list()
    for i in range(df.shape[0]):
        wakati = df["text_wakati"].values[i]
        del_one = ""
        for idx in range(len(wakati)):
            if len(str(wakati[idx])) > 1:
                del_one += (wakati[idx]+" ")
        delonelist.append(np.array(del_one))
    df["text_del_one"] = np.array(delonelist)
    return df

In [7]:
#trainデータの読み込み
train_df = pd.read_csv('./data/train.csv')
#train_df = train_df.head(100)
train_df = add_flags_nums(train_df)

c:\users\井原輝人\desktop\sorcecode\twitter-compe\venv\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
c:\users\井原輝人\desktop\sorcecode\twitter-compe\venv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
train_df = cleaning(train_df)

In [9]:
train_df = wakati(train_df)
train_df = del_one(train_df)

In [10]:
train_df.to_csv("./arranged_data/train2.csv")

In [11]:
train_df.columns

Index(['id', 'flg', 'keyword', 'location', 'text', 'is_url', 'is_location',
       'num_emoji', 'num_reply', 'num_hash', 'text_length', 'num_kusa',
       'is_date', 'is_time', 'text_wakati', 'text_del_one'],
      dtype='object')

In [28]:
X_df_train = train_df[["is_url","is_location","num_emoji","num_reply","num_hash","text_length","num_kusa","is_date","is_time","text_del_one"]]
Y_df_train = train_df["flg"].astype(np.int64)
X_train =  X_df_train.values
y_train = Y_df_train.values

In [27]:
X_df_test = test_df[["is_url","is_location","num_emoji","num_reply","num_hash","text_length","num_kusa","is_date","is_time","text_del_one"]]

X_test =  X_df_test.values


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = CountVectorizer()
# トレーニング用データにCountVectorizerを適用
X_train = cv.fit_transform(X_train[:,9])
# バリデーション用データにCountVectorizerを適用
X_test = cv.transform(X_test[:,9])

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500,
                                    random_state=1)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score, recall_score,f1_score
import seaborn as sns
def result_heatmap(Y_test,Y_pred):
    print("正解率:"+str(accuracy_score(Y_test, Y_pred)))
    print("適合率:"+str(recall_score(Y_test,Y_pred)))
    print("再現率:"+str(precision_score(Y_test,Y_pred)))
    print("F値:"+str(f1_score(Y_test, Y_pred)))  
    cm = confusion_matrix(Y_test, Y_pred)
    sns.heatmap(cm,annot=True, cmap='Blues')

In [32]:
#普通にやると全部違う方に分類される
y_pred = model.predict(X_test)
ans_df=pd.DataFrame()
ans_df["flg"] = y_pred
ans_df.to_csv("./data/ans_df.csv",index=False)

In [25]:

test_df = pd.read_csv("./data/test.csv")
test_df = add_flags_nums(test_df)
test_df = cleaning(test_df)
test_df = wakati(test_df)
test_df = del_one(test_df)


c:\users\井原輝人\desktop\sorcecode\twitter-compe\venv\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
c:\users\井原輝人\desktop\sorcecode\twitter-compe\venv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
test_df.to_csv("./arranged_data/test2.csv")